## Dataset Creation

In [1]:
import pandas as pd
from datasets import Dataset, DatasetDict
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split

/Users/alessandropranzo/.pyenv/versions/3.10.15/envs/nlp_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def create_huggingface_dataset(train_data_folder):
    train_data = []
    test_data = []

    for file_name in tqdm(os.listdir(train_data_folder)):
        df = pd.read_csv(os.path.join(train_data_folder, file_name))
        df.drop(columns=['MatchID', 'PeriodID', 'Timestamp'], inplace=True, errors='ignore')
        #Here we split the dataset so to take 20% of the data for each period in each match
        for id_val, group in df.groupby("ID"):
            train_group, test_group = train_test_split(group, test_size=0.2, random_state=42)
            train_data.append(train_group)
            test_data.append(test_group)
    
    train_df = pd.concat(train_data).reset_index(drop=True)
    test_df = pd.concat(test_data).reset_index(drop=True)

    train_df.rename(columns={"EventType": "labels"}, inplace=True)
    test_df.rename(columns={"EventType": "labels"}, inplace=True)

    # Convert to HuggingFace Dataset
    train_dataset = Dataset.from_pandas(train_df)
    test_dataset = Dataset.from_pandas(test_df)

    # Create DatasetDict
    dataset = DatasetDict({
        "train": train_dataset,
        "test": test_dataset
    })
    
    return dataset

In [3]:
dataset = create_huggingface_dataset("cleaned_data/train_data")

100%|██████████| 16/16 [00:03<00:00,  5.16it/s]


In [4]:
dataset.shape

{'train': (2309732, 3), 'test': (578525, 3)}

In [5]:
dataset["train"][0]

{'ID': '2_0',
 'labels': 0,
 'Tweet': 'the socceroos are confident going up against esp! check this out user:'}

# Model Setup

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, DistilBertModel
import evaluate
import numpy as np

import torch
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

Using device: mps


In [7]:
trial = DistilBertModel.from_pretrained("distilbert-base-uncased")

In [8]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [9]:
def preprocess_function(examples):
    return tokenizer(examples["Tweet"], truncation=True)

In [10]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map: 100%|██████████| 578525/578525 [00:07<00:00, 77429.61 examples/s]


In [11]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [12]:
def count_parameters(model):
    """Helper function to count number of parameters, trainable, non-trainable and total."""
    trainable_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
    non_trainable_parameters = sum(p.numel() for p in model.parameters() if not p.requires_grad)
    total_parameters = trainable_parameters + non_trainable_parameters
    print(f"Trainable parameters: {trainable_parameters}")
    print(f"Non-trainable parameters: {non_trainable_parameters}")
    print(f"Total parameters: {total_parameters}")
    return trainable_parameters, non_trainable_parameters, total_parameters

In [13]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2).to(device)

for param in model.base_model.parameters():
    param.requires_grad = False

for param in model.distilbert.transformer.layer[-1].parameters():
    param.requires_grad = True

for param in model.pre_classifier.parameters():
    param.requires_grad = True

for param in model.classifier.parameters():
    param.requires_grad = True

count_parameters(model)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters: 7680002
Non-trainable parameters: 59275008
Total parameters: 66955010


(7680002, 59275008, 66955010)

In [14]:
training_args = TrainingArguments(
    output_dir="model_output",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    use_mps_device=True,
)

/Users/alessandropranzo/.pyenv/versions/3.10.15/envs/nlp_env/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/Users/alessandropranzo/.pyenv/versions/3.10.15/envs/nlp_env/lib/python3.10/site-packages/transformers/training_args.py:2248: UserWarning: `use_mps_device` is deprecated and will be removed in version 5.0 of 🤗 Transformers. `mps` device will be used by default if available similar to the way `cuda` device is used.Therefore, no action from user is required. 
  warnings.warn(


In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/var/folders/14/_5lf5fw13cldzrf7ntwtb8080000gn/T/ipykernel_1545/552906714.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [16]:
trainer.train()

  0%|          | 503/288718 [00:27<3:54:13, 20.51it/s]

{'loss': 0.6806, 'grad_norm': 1.5376558303833008, 'learning_rate': 1.9965364126933547e-05, 'epoch': 0.0}


  0%|          | 1003/288718 [00:53<4:25:25, 18.07it/s]

{'loss': 0.6698, 'grad_norm': 1.0807185173034668, 'learning_rate': 1.9930728253867096e-05, 'epoch': 0.01}


  1%|          | 1503/288718 [01:19<4:18:45, 18.50it/s]

...

                                                         
100%|██████████| 288718/288718 [6:33:17<00:00, 12.39it/s]

{'eval_loss': 0.6259598135948181, 'eval_accuracy': 0.6321109718681128, 'eval_runtime': 1802.2599, 'eval_samples_per_second': 321.0, 'eval_steps_per_second': 20.063, 'epoch': 2.0}


100%|██████████| 288718/288718 [6:33:18<00:00, 12.23it/s]

{'train_runtime': 23598.5489, 'train_samples_per_second': 195.752, 'train_steps_per_second': 12.235, 'train_loss': 0.6329476171114634, 'epoch': 2.0}


TrainOutput(global_step=288718, training_loss=0.6329476171114634, metrics={'train_runtime': 23598.5489, 'train_samples_per_second': 195.752, 'train_steps_per_second': 12.235, 'total_flos': 4.302666124454362e+16, 'train_loss': 0.6329476171114634, 'epoch': 2.0})